# Basic Imports & Sqlite Setup

In [2]:
# Basic Imports
from sqlalchemy import create_engine, text
import pandas as pd
import sqlite3 as sql3

# Importing Processed Data CSV
laptop_data = pd.read_csv("./../Resources/laptop_data.csv")

# Dropping the index column included in the csv
laptop_data = laptop_data.rename(columns={'Unnamed: 0' : "Laptop_Index"})

In [3]:
# Processing the "ScreenResolution" column so that it just stores the resolution (and no other additional
# information like display type or resolution names (i.e. "Full HD" or "4k")
laptop_data["ScreenResolution"] = laptop_data["ScreenResolution"].apply(lambda x: x.split(" ")[-1])
resolutions = laptop_data["ScreenResolution"].apply(lambda x : x.split("x"))

laptop_screen_widths = resolutions.apply(lambda x : x[0])
laptop_screen_heights = resolutions.apply(lambda x : x[1])

laptop_data["ScreenWidth"] = laptop_screen_widths
laptop_data["ScreenHeight"] = laptop_screen_heights

del laptop_data["ScreenResolution"]

laptop_data.head()

,Laptop_Index,Company,TypeName,Inches,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,ScreenWidth,ScreenHeight
0,0,Apple,Ultrabook,13.3,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832,2560,1600
1,1,Apple,Ultrabook,13.3,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232,1440,900
2,2,HP,Notebook,15.6,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000,1920,1080
3,3,Apple,Ultrabook,15.4,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360,2880,1800
4,4,Apple,Ultrabook,13.3,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080,2560,1600


In [4]:
# Separating the CPU manufacturer from the CPU model name from the speed of the CPU
laptop_data["Cpu"] = laptop_data["Cpu"].apply(lambda x: x.split(" "))

cpu_manufacturer = laptop_data["Cpu"].apply(lambda x : x[0])
cpu_speed = laptop_data["Cpu"].apply(lambda x : x[-1][:-3])
cpu_model = laptop_data["Cpu"].apply(lambda x : " ".join(x[1:-1][0:2]))

laptop_data["CpuManufacturer"] = cpu_manufacturer
laptop_data["CpuSpeed"] = cpu_speed
laptop_data["CpuModel"] = cpu_model

del laptop_data["Cpu"]

In [5]:
# Processing the storage component of the dataset (which is mislabeled as "memory" in the dataset)
def storage_processor(storage_input):
    output_storage_modes = []

    if "+" in storage_input:
        storage_modes = storage_input.split(" + ")

        for storage_mode in storage_modes:
            modifier = 1

            if "TB" in storage_mode:
                modifier = 1000

            split_storage_mode = storage_mode.strip().split(" ")

            storage_amount = float(split_storage_mode[0][0:-2]) * modifier
            storage_type = " ".join(split_storage_mode[1:])

            output_storage_modes.append([storage_amount, storage_type])
    else:
        modifier = 1

        if "TB" in storage_input:
            modifier = 1000

        split_storage_mode = storage_input.split(" ")

        storage_amount = float(split_storage_mode[0][0:-2]) * modifier
        storage_type = " ".join(split_storage_mode[1:])
        
        output_storage_modes.append([storage_amount, storage_type])

    return output_storage_modes

laptop_data["Memory"] = laptop_data["Memory"].apply(storage_processor)

primary_storage_amount = laptop_data["Memory"].apply(lambda x : x[0][0])
primary_storage_type = laptop_data["Memory"].apply(lambda x : x[0][1])

secondary_storage_amount = laptop_data["Memory"].apply(lambda x : x[1][0] if len(x) > 1 else 0)
secondary_storage_type = laptop_data["Memory"].apply(lambda x : x[1][1] if len(x) > 1 else "None")

laptop_data["PrimaryStorageAmount"] = primary_storage_amount
laptop_data["PrimaryStorageType"] = primary_storage_type

laptop_data["SecondaryStorageAmount"] = secondary_storage_amount
laptop_data["SecondaryStorageType"] = secondary_storage_type

del laptop_data["Memory"]

In [6]:
# Processing the memory ("RAM" to differentiate it from the incorrectly labeled "Memory" column in the
# dataset that actually stores data on the laptops storage)
laptop_data["Ram"] = laptop_data["Ram"].apply(lambda x: x[0:-2])
laptop_data = laptop_data.rename(columns={"Ram" : "Memory"})

In [7]:
# Renaming "OpSys" to "OperatingSystem"
laptop_data = laptop_data.rename(columns={"OpSys" : "OperatingSystem"})

In [8]:
# Converting the weight column in the dataset to so that it no longer contains a unit (so it can be read as a numeric value)
laptop_data["Weight"] = laptop_data["Weight"].apply(lambda x : x[0 : -2])

In [9]:
laptop_data.head()

,Laptop_Index,Company,TypeName,Inches,Memory,Gpu,OperatingSystem,Weight,Price,ScreenWidth,ScreenHeight,CpuManufacturer,CpuSpeed,CpuModel,PrimaryStorageAmount,PrimaryStorageType,SecondaryStorageAmount,SecondaryStorageType
0,0,Apple,Ultrabook,13.3,8,Intel Iris Plus Graphics 640,macOS,1.37,71378.6832,2560,1600,Intel,2.3,Core i5,128.0,SSD,0.0,None
1,1,Apple,Ultrabook,13.3,8,Intel HD Graphics 6000,macOS,1.34,47895.5232,1440,900,Intel,1.8,Core i5,128.0,Flash Storage,0.0,None
2,2,HP,Notebook,15.6,8,Intel HD Graphics 620,No OS,1.86,30636.0000,1920,1080,Intel,2.5,Core i5,256.0,SSD,0.0,None
3,3,Apple,Ultrabook,15.4,16,AMD Radeon Pro 455,macOS,1.83,135195.3360,2880,1800,Intel,2.7,Core i7,512.0,SSD,0.0,None
4,4,Apple,Ultrabook,13.3,8,Intel Iris Plus Graphics 650,macOS,1.37,96095.8080,2560,1600,Intel,3.1,Core i5,256.0,SSD,0.0,None


In [10]:
del laptop_data["Laptop_Index"]

In [11]:
company_counts = laptop_data["Company"].value_counts()
companies_to_remove = company_counts[company_counts < 100].index

# Replace in dataframe
for company in companies_to_remove:
    laptop_data['Company'] = laptop_data['Company'].replace(company,"Other")

# Check to make sure replacement was successful
laptop_data['Company'].value_counts()

Company
Dell      297
Lenovo    297
HP        274
Other     174
Asus      158
Acer      103
Name: count, dtype: int64

In [12]:
type_counts = laptop_data["TypeName"].value_counts()
types_to_remove = type_counts[type_counts < 100].index

# Replace in dataframe
for type in types_to_remove:
    laptop_data['TypeName'] = laptop_data['TypeName'].replace(type,"Other")

# Check to make sure replacement was successful
laptop_data['TypeName'].value_counts()

TypeName
Notebook              727
Gaming                205
Ultrabook             196
2 in 1 Convertible    121
Other                  54
Name: count, dtype: int64

In [13]:
gpu_counts = laptop_data["Gpu"].value_counts()
gpus_to_remove = gpu_counts[gpu_counts < 15].index

# Replace in dataframe
for gpu in gpus_to_remove:
    laptop_data['Gpu'] = laptop_data['Gpu'].replace(gpu,"Other")

# Check to make sure replacement was successful
laptop_data['Gpu'].value_counts()

Gpu
Other                         319
Intel HD Graphics 620         281
Intel HD Graphics 520         185
Intel UHD Graphics 620         68
Nvidia GeForce GTX 1050        66
Nvidia GeForce GTX 1060        48
Nvidia GeForce 940MX           43
AMD Radeon 530                 41
Intel HD Graphics 500          39
Intel HD Graphics 400          37
Nvidia GeForce GTX 1070        30
Intel HD Graphics              29
Nvidia GeForce GTX 1050 Ti     28
AMD Radeon R5 M430             22
Nvidia GeForce 930MX           20
AMD Radeon 520                 17
Nvidia GeForce MX150           15
Intel HD Graphics 515          15
Name: count, dtype: int64

In [14]:
os_counts = laptop_data["OperatingSystem"].value_counts()
os_to_remove = os_counts[os_counts < 10].index

# Replace in dataframe
for os in os_to_remove:
    laptop_data['OperatingSystem'] = laptop_data['OperatingSystem'].replace(os,"Other")

# Check to make sure replacement was successful
laptop_data['OperatingSystem'].value_counts()

OperatingSystem
Windows 10    1072
No OS           66
Linux           62
Windows 7       45
Chrome OS       27
Other           18
macOS           13
Name: count, dtype: int64

In [15]:
laptop_data = laptop_data[(laptop_data["CpuManufacturer"] == "Intel") | (laptop_data["CpuManufacturer"] == "AMD")]

In [16]:
cpu_counts = laptop_data["CpuModel"].value_counts()
cpu_to_remove = cpu_counts[cpu_counts < 10].index

# Replace in dataframe
for cpu in cpu_to_remove:
    laptop_data['CpuModel'] = laptop_data['CpuModel'].replace(cpu,"Other")

# Check to make sure replacement was successful
laptop_data['CpuModel'].value_counts()

CpuModel
Core i7           527
Core i5           423
Core i3           136
Celeron Dual       80
Other              78
Pentium Quad       27
Core M             19
A9-Series 9420     12
Name: count, dtype: int64

In [17]:
laptop_data["SecondaryStorageType"].value_counts()

SecondaryStorageType
None      1094
HDD        202
SSD          4
Hybrid       2
Name: count, dtype: int64

In [18]:
print(len(laptop_data.columns))

17


In [ ]:
# Creating the database sqlite file to insert the data into from the csv
database_path = 'laptop_data_database.sqlite'

db_con = sql3.connect(database_path)
cursor = db_con.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS Laptops (
COMPANY TEXT NOT NULL,
TYPENAME TEXT NOT NULL,
INCHES FLOAT NOT NULL,
MEMORY INT NOT NULL,
GPU TEXT NOT NULL,
OPERATINGSYSTEM TEXT NOT NULL,
WEIGHT FLOAT NOT NULL,
PRICE FLOAT NOT NULL,
SCREENWIDTH INT NOT NULL,
SCREENHEIGHT INT NOT NULL,
CPUMANUFACTURER TEXT NOT NULL,
CPUSPEED FLOAT NOT NULL,
CPUMODEL TEXT NOT NULL,
PRIMARYSTORAGEAMOUNT FLOAT NOT NULL,
PRIMARYSTORAGETYPE TEXT NOT NULL,
SECONDARYSTORAGEAMOUNT FLOAT,
SECONDARYSTORAGETYPE FLOAT
)''')

db_con.commit()
db_con.close()

In [ ]:
# Inputting the laptop data into the sqlite file we created above
engine = create_engine(f"sqlite:///{database_path}", echo=True)
connection = engine.connect()

laptop_data.to_sql("Laptops", con=connection, if_exists="append", index=False)
laptop_data.to_csv("laptop_data.csv")

connection.commit()
connection.close()
engine.dispose()

2025-02-10 20:10:29,406 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-10 20:10:29,413 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Laptops")
2025-02-10 20:10:29,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-10 20:10:29,444 INFO sqlalchemy.engine.Engine INSERT INTO "Laptops" ("Company", "TypeName", "Inches", "Memory", "Gpu", "OperatingSystem", "Weight", "Price", "ScreenWidth", "ScreenHeight", "CpuManufacturer", "CpuSpeed", "CpuModel", "PrimaryStorageAmount", "PrimaryStorageType", "SecondaryStorageAmount", "SecondaryStorageType") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-02-10 20:10:29,445 INFO sqlalchemy.engine.Engine [generated in 0.01874s] [('Other', 'Ultrabook', 13.3, '8', 'Other', 'macOS', '1.37', 71378.6832, '2560', '1600', 'Intel', '2.3', 'Core i5', 128.0, 'SSD', 0.0, 'None'), ('Other', 'Ultrabook', 13.3, '8', 'Other', 'macOS', '1.34', 47895.5232, '1440', '900', 'Intel', '1.8', 'Core i5', 128.0, 'Flash Storage', 0.0, 'None'), ('HP',